# GCN using NumPy

Based on "Graph Convolutional Networks using only Numpy" video by WelcomeAIOverlords on YouTube

In [1]:
import numpy as np
import networkx as nx
from scipy.linalg import sqrtm
# import matplotlib.pyplot as plt

Loading in graph and its adjacency matrix

In [2]:
G = nx.karate_club_graph() #graph
A = nx.to_numpy_array(G) #adj matrix
A

array([[0., 4., 5., ..., 2., 0., 0.],
       [4., 0., 6., ..., 0., 0., 0.],
       [5., 6., 0., ..., 0., 2., 0.],
       ...,
       [2., 0., 0., ..., 0., 4., 4.],
       [0., 0., 2., ..., 4., 0., 5.],
       [0., 0., 0., ..., 4., 5., 0.]])

Getting community labels from Karate club

In [3]:
club_labels = nx.get_node_attributes(G, 'club')
club_labels

{0: 'Mr. Hi',
 1: 'Mr. Hi',
 2: 'Mr. Hi',
 3: 'Mr. Hi',
 4: 'Mr. Hi',
 5: 'Mr. Hi',
 6: 'Mr. Hi',
 7: 'Mr. Hi',
 8: 'Mr. Hi',
 9: 'Officer',
 10: 'Mr. Hi',
 11: 'Mr. Hi',
 12: 'Mr. Hi',
 13: 'Mr. Hi',
 14: 'Officer',
 15: 'Officer',
 16: 'Mr. Hi',
 17: 'Mr. Hi',
 18: 'Officer',
 19: 'Mr. Hi',
 20: 'Officer',
 21: 'Mr. Hi',
 22: 'Officer',
 23: 'Officer',
 24: 'Officer',
 25: 'Officer',
 26: 'Officer',
 27: 'Officer',
 28: 'Officer',
 29: 'Officer',
 30: 'Officer',
 31: 'Officer',
 32: 'Officer',
 33: 'Officer'}

Loading in normalized form of adjacency matrix from Kipf and Welling's GCN paper:

$$\hat{A} = \tilde{D}^{-\frac{1}{2}}\tilde{A}\tilde{D}^{-\frac{1}{2}}$$

where $\tilde{A} = A + I$ introduces self connections and $\hat{A}_{i,j} = \frac{1}{\sqrt{\tilde{d_i}\tilde{d_j}}}\tilde{A}_{i,j}$ 

In [4]:
A_tilde = A = np.eye(G.number_of_nodes())

D_tilde = np.zeros_like(A_tilde)
np.fill_diagonal(D_tilde, np.asarray(A_tilde.sum(axis=1)).flatten())

D_tilde_invroot = np.linalg.inv(sqrtm(D_tilde))

A_hat = D_tilde_invroot @ A_tilde @ D_tilde_invroot

# np.all(D_tilde == np.eye(G.number_of_nodes()))

Introducing node features using the identity as input features for lack of features. This will map each node to a column of learnable parameters in the first layer. 

In [5]:
X = np.eye(G.number_of_nodes())

Helper/utility functions

In [6]:
#function to initialize weights
def glotrot_init(nin, nout):
    sd = np.sqrt(6.0 / (nin + nout))
    return np.random.uniform(-sd, sd, size=(nin, nout))

def xent(pred, labels):
    return -np.log(pred)[np.arange(pred.shpae[0]), np.argmax(labels, axis=1)]

def norm_diff(dW, dW_approx):
    return np.linalg.norm(dW - dW_approx) / (np.linalg.norm(dW) + np.linalg.norm(dW_approx))

#class for Gradient Descent optimize
class GradDescentOptim():
    def __init__(self, lr, wd):
        self.lr = lr
        self.wd = wd
        self._y_pred = None
        self._y_true = None
        self._out = None
        self.bs = None
        self.train_nodes = None
        
    def __call__(self, y_pred, y_true, train_nodes=None):
        self.y_pred = y_pred
        self.y_true = y_true
        
        if train_nodes is None:
            self.train_nodes = np.arange(y_pred.shape[0])
        else:
            self.train_nodes = train_nodes
            
        self.bs = self.train_nodes.shape[0]
        
    @property
    def out(self):
        return self._out
    
    @out.setter
    def out(self, y):
        self._out = y

Creating class representing a single GCN layer. These can then be stacked to create a full GCN model. Kipf and Welling represent a layer by the following:

$$H^{l + 1} = \sigma{(W \hat{A} H^l)}$$

where $l$ is the layer number, $H^0$ represents the input features, $\hat{A}H^l$ represents the GCN's message passing, $W$ represents a learned weight matrix, and $\sigma{(\cdot)}$ represents a nonlinear activation function

In [7]:
class GCNLayer():
    def __init__(self, n_inputs, n_outputs, activation=None, name=""):
        self.n_inputs = n_inputs
        self.n_outputs = n_outputs
        self.W = glorot_init(self.n_outputs, self.n_inputs) 
        self.activation = activation
        self.name = name

    def __repr__(self):
        return f"GCN: W{('_' + self.name) if self.name else ''}"
    
    def forward(self, A, X, W=None):
        """
        A = bs x bs normalized adjacency matrix (bs = batch size)
        X = bs x D input features (D = length of input features)
        """
        self._X = (A @ X).T #calculating gradients (D x bs)
        
        if W is None:
            W = self.W

        H = W @ self._X #(h x D) * (D x bs) = (h x bs)

        if self.activation is not None:
            H = self.activation(H)
    
        self._H = H #(h x bs)

        return self._H.T #(bs x h)
    
    def backward(self, optim, update=True):
        dtanh = 1 - np.asarray(self._H.T)**2 #(bs x out_dim)
        
        pass

left off around 5:30